In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
tf.__version__

'2.3.1'

In [3]:
! pip install split-folders

## Easier way to split into train , test data

In [4]:
import splitfolders

# train, test split
splitfolders.ratio('./data/brain_tumor_dataset/', output="./data/brain_tumor_dataset_split", ratio=(0.7, 0.3))

Copying files: 253 files [00:00, 936.55 files/s]


## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [5]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('./data/brain_tumor_dataset_split/train',
                                                 target_size = (64,64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 176 images belonging to 2 classes.


In [6]:
training_set.class_indices

{'no': 0, 'yes': 1}

In [7]:
yes=np.count_nonzero(training_set.classes)
print("Yes:",yes)
print("No:",176-yes)

Yes: 108
No: 68


### Preprocessing the Test set

In [8]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('data/brain_tumor_dataset_split/val/',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 77 images belonging to 2 classes.


In [9]:
yes_test=np.count_nonzero(test_set.classes)
print("Yes:",yes_test)
print("No:",77-yes_test)

Yes: 47
No: 30


## Part 2 - Building the CNN

### Initialising the CNN

In [10]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [11]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=7, activation='relu', input_shape=[64,64,3]))

### Step 2 - Pooling

In [12]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [13]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [14]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [15]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Step 5 - Output Layer

In [16]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [17]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Summary of the model

In [18]:
cnn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 58, 58, 32)        4736      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 29, 29, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 27, 27, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 5408)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               692352    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 1

### Training the CNN on the Training set and evaluating it on the Test set

In [19]:
cnn.fit(x = training_set, epochs = 121 ,validation_data=test_set)

Epoch 1/121
6/6 [==============================] - 1s 207ms/step - loss: 0.6577 - accuracy: 0.6307 - val_loss: 0.5877 - val_accuracy: 0.6494
Epoch 2/121
6/6 [==============================] - 1s 172ms/step - loss: 0.5613 - accuracy: 0.7216 - val_loss: 0.5488 - val_accuracy: 0.7273
Epoch 3/121
6/6 [==============================] - 1s 151ms/step - loss: 0.5673 - accuracy: 0.7102 - val_loss: 0.5230 - val_accuracy: 0.7403
Epoch 4/121
6/6 [==============================] - 1s 163ms/step - loss: 0.5818 - accuracy: 0.6818 - val_loss: 0.5225 - val_accuracy: 0.7403
Epoch 5/121
6/6 [==============================] - 1s 161ms/step - loss: 0.5500 - accuracy: 0.7216 - val_loss: 0.5085 - val_accuracy: 0.7662
Epoch 6/121
6/6 [==============================] - 1s 144ms/step - loss: 0.5159 - accuracy: 0.7614 - val_loss: 0.4801 - val_accuracy: 0.7922
Epoch 7/121
6/6 [==============================] - 1s 144ms/step - loss: 0.4733 - accuracy: 0.7784 - val_loss: 0.5208 - val_accuracy: 0.7662
Epoch 8/121
6

Epoch 59/121
6/6 [==============================] - 1s 173ms/step - loss: 0.1304 - accuracy: 0.9545 - val_loss: 0.8007 - val_accuracy: 0.8182
Epoch 60/121
6/6 [==============================] - 1s 182ms/step - loss: 0.1353 - accuracy: 0.9489 - val_loss: 0.8189 - val_accuracy: 0.7922
Epoch 61/121
6/6 [==============================] - 1s 226ms/step - loss: 0.1611 - accuracy: 0.9489 - val_loss: 0.7388 - val_accuracy: 0.7922
Epoch 62/121
6/6 [==============================] - 1s 159ms/step - loss: 0.1520 - accuracy: 0.9375 - val_loss: 0.9091 - val_accuracy: 0.7792
Epoch 63/121
6/6 [==============================] - 1s 201ms/step - loss: 0.1160 - accuracy: 0.9545 - val_loss: 0.9190 - val_accuracy: 0.7922
Epoch 64/121
6/6 [==============================] - 1s 199ms/step - loss: 0.1362 - accuracy: 0.9545 - val_loss: 0.8864 - val_accuracy: 0.7922
Epoch 65/121
6/6 [==============================] - 2s 252ms/step - loss: 0.1201 - accuracy: 0.9545 - val_loss: 0.8792 - val_accuracy: 0.8052
Epoch 

6/6 [==============================] - 1s 162ms/step - loss: 0.0407 - accuracy: 0.9830 - val_loss: 1.5491 - val_accuracy: 0.7662
Epoch 117/121
6/6 [==============================] - 1s 160ms/step - loss: 0.0197 - accuracy: 0.9943 - val_loss: 1.3985 - val_accuracy: 0.7792
Epoch 118/121
6/6 [==============================] - 1s 166ms/step - loss: 0.0228 - accuracy: 1.0000 - val_loss: 1.3416 - val_accuracy: 0.8052
Epoch 119/121
6/6 [==============================] - 1s 163ms/step - loss: 0.0194 - accuracy: 0.9886 - val_loss: 1.3357 - val_accuracy: 0.8182
Epoch 120/121
6/6 [==============================] - 1s 208ms/step - loss: 0.0346 - accuracy: 0.9830 - val_loss: 1.3644 - val_accuracy: 0.8052
Epoch 121/121
6/6 [==============================] - 1s 214ms/step - loss: 0.0315 - accuracy: 0.9830 - val_loss: 1.4342 - val_accuracy: 0.7922


### Train accuracy

In [20]:
cnn.evaluate(training_set)

6/6 [==============================] - 1s 107ms/step - loss: 0.0177 - accuracy: 1.0000


[0.017665710300207138, 1.0]

### Test accuracy 

In [21]:
cnn.evaluate(test_set)

3/3 [==============================] - 0s 47ms/step - loss: 1.4342 - accuracy: 0.7922


[1.4342478513717651, 0.7922077775001526]

## Part 4 - Making single predictions

In [22]:
import numpy as np
from tensorflow.keras.preprocessing import image


from IPython.display import display, Image
display(Image(filename='brain_tumor_dataset_splitted1/val/no/46 no.jpg'))
test_image = image.load_img('brain_tumor_dataset_splitted1/val/no/46 no.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
if result[0][0] == 1:
  prediction = 'Yes'
else:
  prediction = 'No'

print("Actual: No")
print("Predicted:", prediction)

FileNotFoundError: [Errno 2] No such file or directory: 'brain_tumor_dataset_splitted1/val/no/46 no.jpg'

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image


from IPython.display import display, Image
display(Image(filename='brain_tumor_dataset_splitted1/val/no/22 no.jpg'))
test_image = image.load_img('brain_tumor_dataset_splitted1/val/no/22 no.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
  prediction = 'Yes'
else:
  prediction = 'No'

print("Actual: No")
print("Predicted:", prediction)

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image


from IPython.display import display, Image
display(Image(filename='brain_tumor_dataset_splitted1/val/yes/Y147.JPG'))
test_image = image.load_img('brain_tumor_dataset_splitted1/val/yes/Y147.JPG', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
  prediction = 'Yes'
else:
  prediction = 'No'

print("Actual: Yes")
print("Predicted:", prediction)

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image


from IPython.display import display, Image
display(Image(filename='brain_tumor_dataset_splitted1/val/yes/Y3.jpg'))
test_image = image.load_img('brain_tumor_dataset_splitted1/val/yes/Y3.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
  prediction = 'Yes'
else:
  prediction = 'No'

print("Actual: Yes")
print("Predicted:", prediction)